<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>

# Parsing Output

Let's set up a Chat Model:

In [197]:
from langchain.prompts import PromptTemplate, SystemMessagePromptTemplate,ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
api_key = open("C://Users//Marcial//Desktop//desktop_openai.txt").read()
model = ChatOpenAI(openai_api_key=api_key)

## List Parsing 

In [198]:
from langchain.output_parsers import CommaSeparatedListOutputParser

In [199]:
output_parser = CommaSeparatedListOutputParser()

In [200]:
format_instructions = output_parser.get_format_instructions()

In [201]:
print(format_instructions)

Your response should be a list of comma separated values, eg: `foo, bar, baz`


In [203]:
reply = "one, two, three"
output_parser.parse("one, two, three")

['one', 'two', 'three']

In [204]:
human_template = '{request} {format_instructions}'
human_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [205]:
chat_prompt = ChatPromptTemplate.from_messages([human_prompt])

chat_prompt.format_prompt(request="give me 5 characteristics of dogs",
                   format_instructions = output_parser.get_format_instructions())

ChatPromptValue(messages=[HumanMessage(content='give me 5 characteristics of dogs Your response should be a list of comma separated values, eg: `foo, bar, baz`', additional_kwargs={}, example=False)])

In [206]:
request = chat_prompt.format_prompt(request="give me 5 characteristics of dogs",
                   format_instructions = output_parser.get_format_instructions()).to_messages()

In [207]:
result = model(request)

In [208]:
result.content

'loyal, friendly, playful, protective, trainable'

In [209]:
# Convert to desired output:
output_parser.parse(result.content)

['loyal', 'friendly', 'playful', 'protective', 'trainable']

## Datetime Parser 

In [144]:
from langchain.output_parsers import DatetimeOutputParser

In [145]:
output_parser = DatetimeOutputParser()

In [146]:
print(output_parser.get_format_instructions())

Write a datetime string that matches the 
            following pattern: "%Y-%m-%dT%H:%M:%S.%fZ". Examples: 0976-07-27T05:21:47.523819Z, 1255-06-12T16:16:30.837257Z, 1427-12-23T03:26:22.664795Z


In [147]:
template_text = "{request}\n{format_instructions}"
human_prompt=HumanMessagePromptTemplate.from_template(template_text)

In [148]:
chat_prompt = ChatPromptTemplate.from_messages([human_prompt])

In [149]:
print(chat_prompt.format(request="When was the 13th Amendment ratified in the US?",
                   format_instructions=output_parser.get_format_instructions()
                   ))

Human: When was the 13th Amendment ratified in the US?
Write a datetime string that matches the 
            following pattern: "%Y-%m-%dT%H:%M:%S.%fZ". Examples: 1519-03-01T21:49:11.978127Z, 0697-08-16T06:05:29.900940Z, 0245-06-28T02:48:29.782887Z


In [150]:
request = chat_prompt.format_prompt(request="What date was the 13th Amendment ratified in the US?",
                   format_instructions=output_parser.get_format_instructions()
                   ).to_messages()

In [151]:
result = model(request,temperature=0)

In [152]:
# Careful with this, it sometimes will include extra information!
result.content

'The 13th Amendment was ratified in the US on December 6, 1865.\n\nThe datetime string that matches the given pattern is: "1865-12-06T00:00:00.000000Z"'

In [153]:
result.content

'The 13th Amendment was ratified in the US on December 6, 1865.\n\nThe datetime string that matches the given pattern is: "1865-12-06T00:00:00.000000Z"'

In [154]:
output_parser.parse(result.content)

OutputParserException: Could not parse datetime string: The 13th Amendment was ratified in the US on December 6, 1865.

The datetime string that matches the given pattern is: "1865-12-06T00:00:00.000000Z"

---

# Methods to Fix Parsing Issues

## Auto-Fix Parser

In [193]:
from langchain.output_parsers import OutputFixingParser

output_parser = DatetimeOutputParser()

misformatted = result.content

In [194]:
misformatted

'The 13th Amendment was ratified in the US on December 6, 1865.\n\nThe datetime string that matches the given pattern is: "1865-12-06T00:00:00.000000Z"'

In [195]:
new_parser = OutputFixingParser.from_llm(parser=output_parser, llm=model)

In [196]:
new_parser.parse(misformatted)

datetime.datetime(1865, 12, 6, 0, 0)

____
### Fixing via System Prompt:

In [136]:
system_prompt = SystemMessagePromptTemplate.from_template("You always reply to questions only in datetime patterns.")
template_text = "{request}\n{format_instructions}"
human_prompt=HumanMessagePromptTemplate.from_template(template_text)

In [137]:
chat_prompt = ChatPromptTemplate.from_messages([system_prompt,human_prompt])

In [138]:
print(chat_prompt.format(request="When was the 13th Amendment ratified in the US?",
                   format_instructions=output_parser.get_format_instructions()
                   ))

System: You always reply to questions only in datetime patterns.
Human: When was the 13th Amendment ratified in the US?
Write a datetime string that matches the 
            following pattern: "%Y-%m-%dT%H:%M:%S.%fZ". Examples: 1796-02-26T04:04:10.673088Z, 0754-04-24T13:43:26.442719Z, 0382-07-21T05:34:03.561213Z


In [139]:
request = chat_prompt.format_prompt(request="What date was the 13th Amendment ratified in the US?",
                   format_instructions=output_parser.get_format_instructions()
                   ).to_messages()

In [140]:
result = model(request,temperature=0)

In [141]:
result.content

'1865-12-06T00:00:00.000000Z'

In [143]:
output_parser.parse(result.content)

datetime.datetime(1865, 12, 6, 0, 0)

Be careful! This could technically be construed as wrong? The full details from Wikipedia:

    27th state to ratify was Georgia: December 6, 1865

    Having been ratified by the legislatures of three-fourths of the states (27 of the 36 states, including those that had been in rebellion), Secretary of State Seward, on December 18, 1865, certified that the Thirteenth Amendment had become valid, to all intents and purposes, as a part of the Constitution.

You also have the issue of states leaving the union, which complicates what a full ratification means at that time. It kind of depends what is meant by the word "ratified"!

## Pydantic JSON Parser
You should also be aware of OpenAI's own JSON offerings (which are still quite new at this time!): https://platform.openai.com/docs/guides/gpt/function-calling


In [53]:
#pip install pydantic

In [55]:
from langchain.output_parsers import PydanticOutputParser

In [64]:
from pydantic import BaseModel, Field

In [69]:
class Scientist(BaseModel):
    
    name: str = Field(description="Name of a Scientist")
    discoveries: list = Field(description="Python list of discoveries")

In [70]:
query = 'Name a famous scientist and a list of their discoveries' 

In [71]:
parser = PydanticOutputParser(pydantic_object=Scientist)

In [73]:
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"title": "Name", "description": "Name of a Scientist", "type": "string"}, "discoveries": {"title": "Discoveries", "description": "Python list of discoveries", "type": "array", "items": {}}}, "required": ["name", "discoveries"]}
```


In [75]:
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

_input = prompt.format_prompt(query="Tell me about a famous scientist")

output = model(_input.to_string())

parser.parse(output)

Scientist(name='Albert Einstein', discoveries=['Theory of Relativity', 'Photoelectric Effect', 'Brownian Motion'])